In [1]:
import pandas as pd
import numpy as np

<img src="./data/table_relationships.JPG" />
<strong>Import game data</strong>

In [3]:
# Import game table data
game_csv = pd.read_csv('./data/game.csv').fillna(np.nan)

# Import team info table data
team_csv = pd.read_csv("./data/team_info.csv").fillna(np.nan)

In [8]:
# Shape of data (number of rows, and columns)
# Get the shape of game csv
print(game_csv.shape)
# Get the shape of team csv
print(team_csv.shape)

(7441, 15)
(33, 6)


In [ ]:
# Why is our dataset interesting to data science

In [ ]:
# Think of interesting data points

# Focus on these two topics
# 1 - Goals scored at a specific venue and performance of home team at that venue
# 2 - Timezone effects on teams performance (goals scored)

# Extra points to investigate if not enough data previously
# - Time spent on ice versus goals scored
# - Team performance based on time between games played (home/away)

# Make graphs based on analysis stuff